In [41]:
# ======================================
# 1. 라이브러리 로드
# ======================================
from neo4j import GraphDatabase
from openai import OpenAI
import json


# ======================================
# 2. Neo4j AuraDB 연결 정보
# ======================================
AURA_URI = "neo4j+s://7984f269.databases.neo4j.io"
AURA_USER = "neo4j"
AURA_PASSWORD = "zsrjxob6URCybiPttGcBDkDwwduQh6M2fFMwsDdZ3_A"

driver = GraphDatabase.driver(
    AURA_URI,
    auth=(AURA_USER, AURA_PASSWORD)
)


# ======================================
# 3. Neo4j 조회 함수
# - Cypher를 그대로 실행
# - RETURN 결과를 JSON(dict) 형태로 수집
# ======================================
def query_neo4j():
    cypher = """
    MATCH (c:Company {company_id: 'SamsungElectronics'})
    MATCH (c)-[:HAS_EVENT]->(e:Event)-[:ON_DATE]->(d:Day {date: '2025.6.11'})
    MATCH (c)-[:HAS_SHAP]->(s:Shap)-[:ON_DATE]->(d)
    RETURN  
        {
            company_id: c.company_id,
            date: d.date,
            y_stock: e.y_stock,
            shap: {
                E1: s.E1,
                E2: s.E2,
                E3: s.E3,
                E4: s.E4,
                E5: s.E5,
                E6: s.E6,
                E7: s.E7
            }
        } AS result
    """

    with driver.session() as session:
        result = session.run(cypher)

        data = []
        for record in result:
            data.append(record["result"])

        return data



# ======================================
# 4. LLM 프롬프트 생성
# ======================================
def build_prompt(graph_json):
    return f"""
너는 금융 애널리스트다.
아래는 특정 날짜와 종목에 대해 Neo4j 그래프 DB에서 추출한
노드와 엣지 정보(JSON)이다.

이 JSON을 해석하여
해당 날짜의 주가 변동 원인을 자연어로 설명하라.

원인의 경우 아래 [원인 - 설명 매칭]을 활용하여 답변하라.

[원인 - 설명 매칭]
E1: 실적/가이던스
E2: 수주/계약
E3: 정책/규제
E4: 제품/기술
E5: 투자/증설
E6: M&A/지분
E7: 사고/리스크

[그래프 데이터(JSON)]
{json.dumps(graph_json, ensure_ascii=False, indent=2)}

[출력 형식 예시]
"2025-01-01 주가 상승은 두산에너빌리티의 E1(실적/가이던스)이
주된 원인이었으며, shap 비율은 0.5로 해당일 주가상승에 가장 큰 영향을 끼쳤다.
이어서 E2(수주/계약)와 E4(제품/기술)도 높은 수치의 shap 비율을 기록했다"
"""


# ======================================
# 5. OpenAI 클라이언트
# ======================================
client = OpenAI()


def generate_natural_language(prompt):
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "너는 금융 데이터 해석에 특화된 애널리스트다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content


# ======================================
# 6. 실행부
# ======================================
if __name__ == "__main__":

    graph_json = query_neo4j()

    print("\n📦 Neo4j 반환 JSON")
    print(json.dumps(graph_json, indent=2, ensure_ascii=False))

    prompt = build_prompt(graph_json)
    answer = generate_natural_language(prompt)

    print("\n📈 LLM 자연어 분석 결과")
    print(answer)



📦 Neo4j 반환 JSON
[
  {
    "date": "2025.6.11",
    "shap": {
      "E5": 0.027090399,
      "E6": 0.002302661,
      "E7": 0.013426386,
      "E1": 0.121451988,
      "E2": 0.060686445,
      "E3": -0.021315971,
      "E4": 0.090924736
    },
    "company_id": "SamsungElectronics",
    "y_stock": 1.1824296276122803
  }
]

📈 LLM 자연어 분석 결과
"2025-06-11 주가 상승은 SamsungElectronics의 E1(실적/가이던스)이
주된 원인이었으며, shap 비율은 0.1215로 해당일 주가 상승에 가장 큰 영향을 끼쳤다.
이어서 E4(제품/기술)와 E2(수주/계약)도 각각 0.0909와 0.0607의 높은 shap 비율을 기록하여 주가 상승에 긍정적인 영향을 미쳤다.
반면, E3(정책/규제)은 -0.0213으로 다소 부정적인 영향을 끼쳤으나 전체 상승세를 막지는 못했다."
